## Task 1 - SQL

### Build SparkSession:

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession 
spark = (SparkSession.builder.appName('SparkSQL')
         .enableHiveSupport() # Hive support is required to CREATE Hive TABLE (AS SELECT)
         .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.0.1") # To read Avro format
         .getOrCreate())

21/10/29 08:15:53 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.122.132 instead (on interface ens33)
21/10/29 08:15:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/eslam/.ivy2/cache
The jars for the packages stored in: /home/eslam/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d13db87-dc81-4f50-9e0e-cb4cb6e7d2de;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.0.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 831ms :: artifacts dl 21ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.0.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

### Read the json file:

In [2]:

df_schema = "Id INT, Model STRING, Year INT, ScreenSize STRING, RAM STRING, HDD STRING, W DOUBLE, D DOUBLE, H DOUBLE, Weight DOUBLE"

In [3]:
df=(spark.read.format('json')
    .schema(df_schema)
    .option('header','true')
    .load('DataFrames_sample.json')
)



### Display the schema:


In [4]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



In [5]:
df.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### Get all the data when "Model" equal "MacBook Pro":




In [6]:
from pyspark.sql.functions import col
df.where(col("Model")=="MacBook Pro").show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### Create TempView:

In [7]:
df.createOrReplaceTempView("df")

### Display "RAM"column and count "RAM" column:

In [8]:
spark.sql("""SELECT RAM
          FROM df 
          """).show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
| 8GB|
|64GB|
+----+



In [9]:
spark.sql("""SELECT COUNT (RAM) 
          FROM df 
          """).show()

+----------+
|count(RAM)|
+----------+
|         4|
+----------+



### Get all columns when "Year" column equal "2015"  

In [10]:
spark.sql("""SELECT * 
          FROM df
          WHERE Year ='2015'
          """).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### Get all when "Model" start with "M":

In [11]:
spark.sql("""SELECT * 
          FROM df
          WHERE Model LIKE 'M%'
          """).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### Get all data when "Model" column equal "MacBook Pro"

In [12]:
spark.sql("""SELECT * 
          FROM df
          WHERE Model = 'MacBook Pro' 
          """).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [13]:
spark.sql("""SELECT * 
          FROM df
          WHERE RAM = '8GB' and Model ='MacBook' 
          """).show()

+---+-------+----+----------+---+---------+-----+----+----+------+
| Id|  Model|Year|ScreenSize|RAM|      HDD|    W|   D|   H|Weight|
+---+-------+----+----------+---+---------+-----+----+----+------+
|  2|MacBook|2016|       12"|8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-------+----+----------+---+---------+-----+----+----+------+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [14]:
spark.sql("""SELECT * 
          FROM df
          WHERE D >= 8 and Model ='iMac' 
          """).show()

+---+-----+----+----------+----+-------+----+---+----+------+
| Id|Model|Year|ScreenSize| RAM|    HDD|   W|  D|   H|Weight|
+---+-----+----+----------+----+-------+----+---+----+------+
|  4| iMac|2017|       27"|64GB|1TB SSD|25.6|8.0|20.3|  20.8|
+---+-----+----+----------+----+-------+----+---+----+------+



## Task 2


### Read "test1" dataset:

In [15]:
test1_shcema = 'Name STRING, age INT, Experience INT, Salary DOUBLE'

In [16]:
df2=(spark.read.format('csv')
    .schema(test1_shcema)
    .option('header','true')
    .load('test1.csv')
)
df2.createOrReplaceTempView("test1")

In [17]:
df2.show()

+---------+---+----------+-------+
|     Name|age|Experience| Salary|
+---------+---+----------+-------+
|    Krish| 31|        10|30000.0|
|Sudhanshu| 30|         8|25000.0|
|    Sunny| 29|         4|20000.0|
|     Paul| 24|         3|20000.0|
|   Harsha| 21|         1|15000.0|
|  Shubham| 23|         2|18000.0|
+---------+---+----------+-------+



In [18]:
df2.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: double (nullable = true)



### Display Salary of the people less than or equal to 20000

In [19]:
spark.sql("""SELECT Name , Salary 
          FROM test1
          WHERE Salary <= 20000 
          """).show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
| Harsha|15000.0|
|Shubham|18000.0|
+-------+-------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [20]:
spark.sql("""SELECT Name , Salary 
          FROM test1
          WHERE Salary <= 20000 and Salary >15000
          """).show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
|Shubham|18000.0|
+-------+-------+



## Task 3 

### Read "test3" dataset:

In [21]:
test3_shcema = 'Name STRING,Departments STRING,salary DOUBLE'
df3=(spark.read.format('csv')
    .schema(test3_shcema)
    .option('header','true')
    .load('test3.csv')
)
df3.createOrReplaceTempView("test3")

### Display dataset

In [22]:
df3.show()

+---------+------------+-------+
|     Name| Departments| salary|
+---------+------------+-------+
|    Krish|Data Science|10000.0|
|    Krish|         IOT| 5000.0|
|   Mahesh|    Big Data| 4000.0|
|    Krish|    Big Data| 4000.0|
|   Mahesh|Data Science| 3000.0|
|Sudhanshu|Data Science|20000.0|
|Sudhanshu|         IOT|10000.0|
|Sudhanshu|    Big Data| 5000.0|
|    Sunny|Data Science|10000.0|
|    Sunny|    Big Data| 2000.0|
+---------+------------+-------+



### Display schema

In [23]:
df3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: double (nullable = true)



### Group by "Name" column and using sum function on "Name" column

In [24]:
spark.sql("""SELECT Name,COUNT(Name) 
          FROM test3
          GROUP BY Name
          """).show()

+---------+-----------+
|     Name|count(Name)|
+---------+-----------+
|Sudhanshu|          3|
|    Sunny|          2|
|    Krish|          3|
|   Mahesh|          2|
+---------+-----------+



### Group by "Name" column and using avg function on "Name" column

In [25]:
spark.sql("""SELECT Name,AVG(Salary) 
          FROM test3
          GROUP BY Name
          """).show()

+---------+------------------+
|     Name|       avg(Salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [26]:
spark.sql("""SELECT Departments,COUNT(Departments) 
          FROM test3
          GROUP BY Departments
          """).show()

+------------+------------------+
| Departments|count(Departments)|
+------------+------------------+
|         IOT|                 2|
|    Big Data|                 4|
|Data Science|                 4|
+------------+------------------+



### Group by "Departments" column and using mean function on "Departments" column:

In [27]:
spark.sql("""SELECT Departments,MEAN(Salary) 
          FROM test3
          GROUP BY Departments
          """).show()

+------------+------------+
| Departments|mean(Salary)|
+------------+------------+
|         IOT|      7500.0|
|    Big Data|      3750.0|
|Data Science|     10750.0|
+------------+------------+



Group by "Departments" column and using count function on "Departments" column:

### Apply agg to using sum function get the total of salaries

In [28]:
spark.sql("""SELECT SUM(Salary) 
          FROM test3
          """).show()

+-----------+
|sum(Salary)|
+-----------+
|    73000.0|
+-----------+



## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [29]:
iti_shcema = 'Ship_name STRING,Cruise_line STRING,Age Double,Tonnage DOUBLE,passengers DOUBLE,length DOUBLE ,cabins DOUBLE,passenger_density DOUBLE,crew DOUBLE'
df4=(spark.read.format('csv')
    .schema(iti_shcema)
    .option('header','true')
    .load('ITI_data.csv')
)
df4.createOrReplaceTempView("iti")

In [30]:
df4.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [31]:
df4.show()

+-----------+-----------+----+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line| Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+----+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara| 6.0|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara| 6.0|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival|26.0|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival|11.0|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival|17.0|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival|22.0|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival|15.0|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carn

In [32]:
# Splitting Data
train_df , test_df = df4.randomSplit([0.8, 0.2] , seed = 42)
print(f"There are {train_df.count()} rows in the training set, and {test_df.count()} in the test set")

There are 133 rows in the training set, and 25 in the test set


### OneHotEncoder 


In [33]:
from pyspark.ml.feature import VectorAssembler

In [34]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

###Use VectorAssembler to merge all columns into one column:

In [48]:
trainDF, testDF = df4.randomSplit([.8,.2],seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 133 rows in the training set, and 25 in the test set


In [35]:

categoricalCols = [field for (field, dataType) in df4.dtypes
                   if dataType == "string"]

In [36]:
categoricalCols

['Ship_name', 'Cruise_line']

In [37]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Ship_name_Index', 'Cruise_line_Index']

In [38]:
oheOutputCols = [x + "_OHE" for x in categoricalCols]
oheOutputCols

['Ship_name_OHE', 'Cruise_line_OHE']

In [39]:
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

In [40]:
numericCols = [field for (field,dataType) in df4.dtypes
              if ((dataType=='double')& (field!='crew'))]
numericCols

['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']

In [41]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Ship_name_OHE',
 'Cruise_line_OHE',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']

In [42]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

### Create a Linear Regression Model 

In [43]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In [44]:
lr = LinearRegression(labelCol='crew',featuresCol='features')


### Creating a Pipeline

In [45]:
pipeline =Pipeline(stages = [stringIndexer,oheEncoder,vecAssembler,lr])

### Model Evaluation

In [47]:
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='rmse')

In [49]:
pipelineModel = pipeline.fit(trainDF)

21/10/29 08:27:16 WARN Instrumentation: [2b64081e] regParam is zero, which might cause numerical instability and overfitting.
21/10/29 08:27:17 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/10/29 08:27:17 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/10/29 08:27:17 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/10/29 08:27:17 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
21/10/29 08:27:17 WARN Instrumentation: [2b64081e] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [50]:
predDF = pipelineModel.transform(testDF)

In [52]:
rmse = regressionEvaluator.evaluate(predDF)
#print("RMSE is {:.1f}".format(rmse))
print(f"RMSE is {rmse:.1f}")

RMSE is 2.9


By Eng. Mostafa Nabieh 
If you have questions, please feel free to ask.

My Email : nabieh.mostafa@yahoo.com

My Whatsapp : +201015197566